In [15]:
%load_ext autoreload
%autoreload 2
%pylab inline
import pandas as pd
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['mod', 'sample', 'f', 'random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [16]:
import sys
sys.path.append("/home/muncok/DL/projects/sv_system/")

import pprint
pp = pprint.PrettyPrinter(indent=4)

from honk_sv import train as train
from honk_sv import model as mod
from honk_sv import dataset as dset
from honk_sv import system as svs
from honk_sv import dataloader as dloader

# Dataframes
command: disjoint speaker split

In [17]:
dataframe_dir = '/home/muncok/DL/dataset/SV_sets/dataframes/'
data_dir = '/home/muncok/DL/dataset/SV_sets/reddots_r2015q4_v1/wav/'
data_df = pd.read_pickle('/home/muncok/DL/dataset/SV_sets/dataframes/Reddots_Dataframe.pkl')

In [18]:
import honk_sv.train as hk
from honk_sv import model as mod
from honk_sv import dataset as dset
from honk_sv import system as svs
from honk_sv import dataloader as dloader

### Dataset split

In [19]:
all_spks = data_df.spk.unique()
all_sents = data_df.sent.unique()
uttrs_counts = data_df.spk.value_counts()

In [9]:
si_spks = list(uttrs_counts[uttrs_counts > 100].index)
sv_spks = list(uttrs_counts[uttrs_counts <= 100].index)

In [10]:
import random 

si_df = data_df[data_df.spk.isin(si_spks)]

# random sampling
si_random_train = si_df.sample(frac=0.8)
si_random_test = si_df.drop(index=si_random_train.index)
si_random_val = si_random_test
print("[random] train:{}, val:{}, test:{}".format(len(si_random_train), len(si_random_val), len(si_random_test)))

[random] train:11747, val:2937, test:2937


In [11]:
import csv
import random

tags = ['train', 'val', 'test']
# choose a si split
sets = {'train':si_random_train, 'val':si_random_val, 'test':si_random_test}
manifest_dir = "../manifests/reddots/"

for tag in tags:
    samples = []
    save_path = os.path.join(manifest_dir,'si_{}_{}_manifest.csv'.format("reddots", tag))
    with open(save_path, 'w') as f:
        for index, row in sets[tag].iterrows():
            file_path = os.path.join(data_dir, row.spk, row.file)
            label = si_spks.index(row.spk)
            sample = ','.join([file_path, str(label)])
            samples.append(sample)
        random.shuffle(samples)
        writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
        writer.writerow(samples)
        print("{} was written".format(save_path))

../manifests/reddots/si_reddots_train_manifest.csv was written
../manifests/reddots/si_reddots_val_manifest.csv was written
../manifests/reddots/si_reddots_test_manifest.csv was written


In [12]:
# sv write_manifest
samples = []
save_path = os.path.join(manifest_dir,'sv_{}_manifest.csv'.format("reddots"))
sv_df = data_df[data_df.spk.isin(sv_spks)]
with open(save_path, 'w') as f:
    for index, row in sv_df.iterrows():
        file_path = os.path.join(data_dir, row.spk, row.file)
        label = sv_spks.index(row.spk)
        sample = ','.join([file_path, str(label)])
        samples.append(sample)
    writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
    writer.writerow(samples)
    print("{} was written".format(save_path))

../manifests/reddots/sv_reddots_manifest.csv was written


# SI Train

In [34]:
model = "SimpleCNN"
dataset = "reddots"

global_config = dict(model=model, dataset=dataset,
                     no_cuda=False,  gpu_no=0,
                     n_epochs=100, batch_size=64,
                     lr=[0.01], schedule=[np.inf], dev_every=1, seed=0, use_nesterov=False,
                     cache_size=32768, momentum=0.9, weight_decay=0.00001,
                     num_workers=16, print_step=100,
                     splice_length=20
                     )

builder = train.ConfigBuilder(
                dset.SpeechDataset.default_config(dataset),
                global_config)
parser = builder.build_argparse()
si_config = builder.config_from_argparse(parser)
si_config['model_class'] = mod.SimpleCNN
train.set_seed(si_config)

In [35]:
si_config['n_labels'] = len(si_spks)
si_config['input_length'] = int(16000*0.2)
manifest_dir = "../manifests/reddots/"
for tag in ['train', 'val', 'test']:
    si_config['{}_manifest'.format(tag)]=os.path.join(manifest_dir,'si_{}_{}_manifest.csv'.format("reddots", tag))

In [40]:
from torch.autograd import Variable
import torch
import torch.nn as nn

si_model = si_config['model_class']()
time_dim = si_config['input_length']//160+1
test_in = Variable(torch.zeros(1,1,time_dim,40), volatile=True)
test_out = si_model(test_in)
si_model.feat_size = test_out.size(1)
si_model.output = nn.Linear(test_out.size(1), si_config["n_labels"])

In [37]:
# si_model.load("models/reddots/si_reddots_frames_res8.pt")
si_config['n_epochs'] = 100
si_config['output_file'] = "../models/reddots/si_reddots_0.2s.pt"
hk.train(si_config, model=si_model)

train step #99 accuracy: 0.171875, loss: 3.1852052211761475
epoch #0, final dev accuracy: 0.3180897215865751
saving best model...
train step #199 accuracy: 0.359375, loss: 2.4508121013641357
train step #299 accuracy: 0.390625, loss: 1.8864681720733643
epoch #1, final dev accuracy: 0.4989631006864989
saving best model...
train step #399 accuracy: 0.4375, loss: 1.7626347541809082
train step #499 accuracy: 0.453125, loss: 1.7073249816894531
epoch #2, final dev accuracy: 0.6330389492753623
saving best model...
train step #599 accuracy: 0.5, loss: 1.2475086450576782
train step #699 accuracy: 0.578125, loss: 1.156954288482666
epoch #3, final dev accuracy: 0.5985650266971777
train step #799 accuracy: 0.6875, loss: 1.3311842679977417
train step #899 accuracy: 0.765625, loss: 0.9914790391921997
epoch #4, final dev accuracy: 0.6462028032036613
saving best model...
train step #999 accuracy: 0.640625, loss: 1.0650572776794434
epoch #5, final dev accuracy: 0.7610483409610984
saving best model...
tr

train step #9099 accuracy: 0.90625, loss: 0.21386298537254333
epoch #49, final dev accuracy: 0.8854107074752098
train step #9199 accuracy: 0.875, loss: 0.44619831442832947
train step #9299 accuracy: 0.875, loss: 0.2925673723220825
epoch #50, final dev accuracy: 0.839810974446987
train step #9399 accuracy: 0.9375, loss: 0.1790803223848343


Process Process-4855:
KeyboardInterrupt
Process Process-4853:
Process Process-4861:
Process Process-4856:
Process Process-4854:
Process Process-4858:
Process Process-4860:
Process Process-4864:
Process Process-4849:
Process Process-4862:
Process Process-4857:
Process Process-4850:
Process Process-4851:
Process Process-4859:
Process Process-4863:
Process Process-4852:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


RuntimeError: DataLoader worker (pid 45900) exited unexpectedly with exit code 1.

Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._tar

  File "/home/muncok/DL/projects/sv_system/honk_sv/dataset.py", line 111, in preprocess
    self._file_cache[example] = data
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 55, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/muncok/DL/projects/sv_system/honk_sv/dataset.py", line 314, in __getitem__
    return self.preprocess(self.audio_files[index]), self.audio_labels[index]
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 55, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/muncok/DL/proj

  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/librosa/core/audio.py", line 122, in load
    frame = util.buf_to_float(frame, dtype=dtype)
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/librosa/core/audio.py", line 122, in load
    frame = util.buf_to_float(frame, dtype=dtype)
  File "/home/muncok/DL/projects/sv_system/honk_sv/dataset.py", line 314, in __getitem__
    return self.preprocess(self.audio_files[index]), self.audio_labels[index]
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/audioread/rawread.py", line 130, in read_data
    data = self._file.readframes(block_samples)
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/librosa/core/spectrum.py", line 1179, in _spectrogram
    S = np.abs(stft(y, n_fft=n_fft, hop_length=hop_length))**power
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/librosa/filters.py", line 251, in mel
    weights[i] = np.maximum(0, np.mi

KeyboardInterrupt
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/librosa/core/time_frequency.py", line 634, in hz_to_mel
    mels[log_t] = min_log_mel + np.log(frequencies[log_t]/min_log_hz) / logstep
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/librosa/core/time_frequency.py", line 675, in mel_to_hz
    freqs = f_min + f_sp * mels
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/scipy/fftpack/basic.py", line 285, in fft
    tmp = work_function(tmp,n,1,0,overwrite_x)
KeyboardInterrupt
KeyboardInterrupt


In [38]:
si_config['train_manifest']

'../manifests/reddots/si_reddots_train_manifest.csv'